# ch3-决策树

## 3.1-信息熵

熵定义为信息的期望值，为了计算熵，我们计算所有类别可能值包含的信息期望值，通过以下公式：
$$
H=-\sum_{i=1}^{n}p(x_i)log_2(p(x_i))
$$
其中，n是分类数目。

#### 函数：计算给定数据集的熵

In [1]:
import math
def calcuEnt(dataset):
    # 样本个数
    numOfDat=len(dataset)
    # 字典： {标签:出现次数}
    labels={}
    for featVec in dataset:
        #dataset最后一列是label列，即类别
        currLabel=featVec[-1]
        if currLabel not in labels.keys():
            labels[currLabel]=0
        labels[currLabel]+=1
    # 计算熵
    ent=0.0
    for key in labels:
        # 概率
        p=float(labels[key])/numOfDat
        # 熵
        ent-=p*math.log(p,2)
    # 返回该数据集的熵
    return ent

## 3-2 划分数据集

对每个特征所划分的数据集的结果计算一次信息熵，选取划分最好的特征。



#### 函数：按照给定特征划分数据集

In [2]:
def splitDat(dataset,axis,value):
    # 划分之后的数据集
    retDat=[]
    for i in range(len(dataSet)):
        # 如果第i个样本的第axis个特征值，为value，则跳过该值
        if dataSet[i][axis]==value: 
            retDat.append(dataSet[i][0:axis] + dataSet[i][axis+1:])
    return retDat

In [3]:
def splitDataSet(dataSet,axis,value): # 按某个特征分类后的数据
    retDataSet=[]
    for featVec in dataSet:
        if featVec[axis]==value:
            reducedFeatVec =featVec[0:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [ ]:
## 3-3 选择最好划分方式下的特征

遍历整个数据集，循环使用计算熵和划分数据集这两个函数，

根据熵计算，找到最好的特征。

def chooseBestFeature(dataset):
    # 样本的特征数,
    # 最后一列是label
    numOfFeature=len(dataset[0])-1
    # 熵
    preEnt=calcuEnt(dataSet)
    # 最优划分下的熵增益值和特征
    bestGain=0.0
    bestFeature=-1
    for i in range(numOfFeature):
        # 保存特征
        featList=[featVec[i] for featVec in dataSet]
        # 去重复
        uniqueList=set(featList)
        # 第i个特征划分，数据集的信息熵 
        newEnt=0.0
        for value in uniqueList:
            # 按照value,进行数据集的划分
            subDat=splitDataSet(dataSet,i,value)
            # 计算每个value划分的prob和信息熵
            p=len(subDat)/float(len(dataSet))
            # 信息熵
            newEnt+=p*calcuEnt(subDat)
        # 判断是否需要为最优划分
        # 信息熵增益
        entGain=preEnt-newEnt
        if (entGain>bestGain):
            bestGain=entGain
            bestFeature=i 
    # 返回bestFeature,bestGain
    return bestFeature

In [4]:
def chooseBestFeature(dataSet):
    numFeatures = len(dataSet[0]) - 1
    # 数据集的原始信息熵
    baseEntropy = calcuEnt(dataSet)
    # 最优的信息增益值, 和最优的Featurn编号
    bestInfoGain, bestFeature = 0.0, -1
    # iterate over all the features
    for i in range(numFeatures):
        # create a list of all the examples of this feature
        # 获取对应的feature下的所有数据
        featList = [example[i] for example in dataSet]
        # get a set of unique values
        # 获取剔重后的集合，使用set对list数据进行去重
        uniqueVals = set(featList)
        # 创建一个临时的信息熵
        newEntropy = 0.0
        # 遍历某一列的value集合，计算该列的信息熵 
        # 遍历当前特征中的所有唯一属性值，对每个唯一属性值划分一次数据集，计算数据集的新熵值，并对所有唯一特征值得到的熵求和。
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            # 计算概率
            prob = len(subDataSet)/float(len(dataSet))
            # 计算信息熵
            newEntropy += prob * calcuEnt(subDataSet)
        # gain[信息增益]: 划分数据集前后的信息变化， 获取信息熵最大的值
        # 信息增益是熵的减少或者是数据无序度的减少。最后，比较所有特征中的信息增益，返回最好特征划分的索引值。
        infoGain = baseEntropy - newEntropy
        if (infoGain > bestInfoGain):
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

## 3-4 递归构建决策树

递归构建决策树：原始数据集，基于最好的feature，划分数据集，由于对应的value可能有多个，所以可能存在大于2个分支的数据集划分。第一次划分之后，数据向下传递到分支的下一个结点，在该节点上继续划分，重复如上流程。

递归结束条件：

+ 分支下所有实例都具有相同分类；
+ 到达叶子结点的数据，属于叶子结点的分类。

输入数据集和标签列表，算法运行中并不需要标签列表，为了给数据一个明确的含义，所以作为一个参数提供。

In [5]:
import operator
# 如果数据集已经处理了所有属性，则采用多数表决的方法决定该叶子结点的分类

def majorCnt(classList):
    classCnt={}
    for ticket in classList:
        # 如果不在字典，则需要初始化为0
        if ticket not in classCnt.keys():
            classCnt[ticket]=0
        classCnt[ticket]+=1
    # 按照出现次数排序
    sortedClassCnt=sorted(classCnt.items(),key=operator.itemgetter(1),reverse=True)
    # 如果数据集已经处理了所有属性，则采用多数表决的方法决定该叶子结点的分类
    return sortedClassCnt[0][0]

In [6]:
def createTree(dataSet,labels):
    # dataset的最后一列是类别
    classList=[example[-1] for example in dataSet]
    # print('start creat node: %d/%d'%(len(dataSet[0]),len(classList)))
    
    # 边界1：类别都相同，则停止划分,直接返回该类别
    if classList.count(classList[0])==len(classList):
        return classList[0]
    # 边界2：如果已经遍历完所有的特征，即没有feature来继续做划分，
    # 则采用投票法，返回出现次数最多的类别。
    if len(dataSet[0])==1:
        return majorCnt(classList)
    
    # 继续划分，构建子节点
    # 最优划分的feature
    bestFeat=chooseBestFeature(dataSet) 
    # 构建样本feature列的信息
    bestFeatLabel=labels[bestFeat]
    # 字典形式保存树
    myTree={bestFeatLabel:{}} 
    # 去掉该列，因为在之后的划分中，
    # 该列不在dataSet中了，需要与labels对应
    tmp=labels[:]
    del(tmp[bestFeat])
    subLabels=tmp[:]
    valueList=[example[bestFeat] for example in dataSet]
    uniqueVals=set(valueList)
    for value in uniqueVals:
        # 复制，使操作不影响到原数据
        
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree

In [7]:
dataSet = [['长', '粗', '男'],
               ['短', '粗', '男'],
               ['短', '粗', '男'],
               ['长', '细', '女'],
               ['短', '细', '女'],
               ['短', '粗', '女'],
               ['长', '粗', '女'],
               ['长', '粗', '女']]
labels = ['头发','声音']  #两个特征
myTree=createTree(dataSet,labels)
myTree

{'声音': {'粗': {'头发': {'长': '女', '短': '男'}}, '细': '女'}}

## 3-5 测试

In [8]:
def predict(tree, labels, testVec):
    """
        tree  决策树模型
        labels Feature标签对应的名称
        testVec    测试输入的数据
    Returns: classLabel    分类的结果值，映射label返回名称
    """
    # 根节点对应的key值，
    # 即第一个feature
    firstStr = list(tree.keys())[0]
    # 根节点对应的value值，
    # 即根结点的子树
    secondDict = tree[firstStr]
    # 判断根节点名称获取根节点在label中的先后顺序，这样就知道输入的testVec怎么开始对照树来做分类
    # 索引
    featIndex = labels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    # 判断分枝是否结束: 判断valueOfFeat是否是dict类型
    if isinstance(valueOfFeat, dict):
        classLabel = predict(valueOfFeat, labels, testVec)
    else:
        classLabel = valueOfFeat
    return classLabel

In [9]:
labe=['头发','声音']
predict(myTree,labe,['长', '粗'])

'女'

## 3-6 存储决策树

In [10]:
def storeTree(tree,filename):
    import pickle
    fw=open(filename,'wb')
    pickle.dump(tree,fw)
    fw.close()


In [11]:
def loadTree(filename):
    import pickle
    fr=open(filename,'rb')
    return pickle.load(fr)

## 3-7 使用决策树预测隐形眼镜类型

In [12]:
fr=open('./lenses.txt')
dataList=[example.strip().split('\t') for example in fr.readlines()]
labelList=['age', 'prescript', 'astigmatic', 'tearRate']
dataList[:3]

[['young', 'myope', 'no', 'reduced', 'no lenses'],
 ['young', 'myope', 'no', 'normal', 'soft'],
 ['young', 'myope', 'yes', 'reduced', 'no lenses']]

In [20]:
import random
dataSet=dataList
random.shuffle(dataSet)
trainDataList=dataSet[:int(0.8*len(dataSet))]
testDataList=dataSet[int(0.8*len(dataSet)):]
print("start create tree")
tree=createTree(trainDataList,labelList)
print("tree is:",tree)

start create tree
tree is: {'tearRate': {'reduced': 'no lenses', 'normal': {'astigmatic': {'no': {'age': {'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}}, 'pre': 'soft', 'young': 'soft'}}, 'yes': 'hard'}}}}


In [31]:
storeTree(tree,"lenses_tree.txt")

In [32]:
myTree=loadTree("./lenses_tree.txt")
myTree

{'tearRate': {'normal': {'astigmatic': {'yes': {'prescript': {'hyper': {'age': {'young': 'hard',
        'presbyopic': 'no lenses',
        'pre': 'no lenses'}},
      'myope': 'hard'}},
    'no': {'age': {'young': 'soft',
      'presbyopic': {'prescript': {'hyper': 'soft', 'myope': 'no lenses'}},
      'pre': 'soft'}}}},
  'reduced': 'no lenses'}}

## 3-8 准确率

In [22]:
def model_test(tree,testDataList, labelList):
    #错误数
    errorCnt = 0
    #遍历测试集
    for i in range(len(testDataList)):
        if testDataList[i][-1] != predict(tree,labelList,testDataList[i][:-1]):
            errorCnt += 1
    #返回准确率
    return 1 - errorCnt / len(testDataList)

In [23]:
#测试准确率
labe=['age', 'prescript', 'astigmatic', 'tearRate']
accur = model_test(tree,testDataList,labe)
print('the accur is:', accur)


the accur is: 0.6
